<a href="https://colab.research.google.com/github/Bianca-Apostolescu/Proiect-Dizertatie/blob/interm/People_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install segmentation_models_pytorch

# Iniatial

In [ ]:
!mkdir Master
!mkdir /content/Master/instances
!mkdir /content/Master/Output

!wget -P Master http://images.cocodataset.org/zips/val2017.zip
!wget -P Master http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip
!wget -P /content/Master/instances http://images.cocodataset.org/annotations/annotations_trainval2017.zip
# !wget -P Master http://images.cocodataset.org/zips/train2017.zip

In [21]:
!unzip /content/Master/val2017.zip -d Master
!unzip /content/Master/panoptic_annotations_trainval2017.zip -d Master
!unzip /content/Master/instances/annotations_trainval2017.zip -d /content/Master/instances
# !unzip /content/Master/train2017.zip -d Master

Archive:  /content/Master/instances/annotations_trainval2017.zip
  inflating: /content/Master/instances/annotations/instances_train2017.json  
  inflating: /content/Master/instances/annotations/instances_val2017.json  
  inflating: /content/Master/instances/annotations/captions_train2017.json  
  inflating: /content/Master/instances/annotations/captions_val2017.json  
  inflating: /content/Master/instances/annotations/person_keypoints_train2017.json  
  inflating: /content/Master/instances/annotations/person_keypoints_val2017.json  


In [22]:
!unzip /content/Master/annotations/panoptic_val2017.zip -d Master
!mv /content/Master/annotations/panoptic_val2017.json /content/Master # move src dest
!mv /content/Master/instances/annotations/instances_val2017.json /content/Master # move src dest
# !unzip /content/Master/panoptic_annotations_trainval2017.zip -d Master

In [9]:
!rm -r /content/Master/val2017.zip
!rm -r /content/Master/panoptic_annotations_trainval2017.zip
# !rm -r /content/Master/train2017.zip

import shutil

shutil.rmtree('/content/Master/annotations', ignore_errors=True)
shutil.rmtree('/content/Master/instances', ignore_errors=True)
shutil.rmtree('/content/Master/__MACOSX', ignore_errors=True)

In [23]:
import shutil

shutil.rmtree('/content/Master/annotations', ignore_errors=True)
shutil.rmtree('/content/Master/instances', ignore_errors=True)

# Libraries

In [1]:
# For managing COCO dataset
from pycocotools.coco import COCO

# For creating and managing folder/ files
import glob
import os
import shutil

# For managing images
from PIL import Image
import skimage.io as io

# Basic libraries
import numpy as np
import pandas as pd
import random
import cv2

# For plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

# For importing models and working with them
## Torch
import torch
import torch.utils.data # for Dataset
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

## Torchvision
import torchvision
from torchvision.transforms import transforms

# For creating train - test splits
from sklearn.model_selection import train_test_split

import pathlib
import pylab
import requests
from io import BytesIO
from pprint import pprint
from tqdm import tqdm
import time
from imutils import paths

# Performance Metrics
from sklearn.metrics import multilabel_confusion_matrix


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


# %matplotlib inline

cpu


# Configuration

In [2]:
data = 'val2017'

# Data and Masks
IMAGE_PATH = os.path.join('/content/Master/', data)
MASK_PATH = os.path.join('/content/Master/', 'panoptic_' + data)

# define the path to the base output directory
BASE_OUTPUT = '/content/Master/Output'

# define the path to the output serialized model, model training plot, and testing image paths
MODEL_PATH = os.path.join(BASE_OUTPUT, "unet_tgs_salt.pth")
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])



# determine if we will be pinning memory during data loading
PIN_MEMORY = True if device == "cuda" else False


# define the number of channels in the input, number of classes, and number of levels in the U-Net model
NUM_CHANNELS = 3
NUM_CLASSES = 2 # person vs Background

# define the input image dimensions
INPUT_IMAGE_WIDTH = 256
INPUT_IMAGE_HEIGHT = 256

# define threshold to filter weak predictions
THRESHOLD = 0.5 # for binary classification

# Functions

## Dataset

In [3]:
class SegmentationDataset(Dataset):
    
    def __init__(self, imagePaths, maskPaths, transforms):
        # store the image and mask filepaths, and augmentation transforms
        self.imagePaths = imagePaths
        self.maskPaths = maskPaths
        self.transforms = transforms
        self.image_list = []
        self.mask_list = []

        # Loading Images

        for index in range (0, len(imagePaths)):
          # grab the image path from the current index
          imagePath = self.imagePaths[index]
          
          # load the image from disk, swap its channels from BGR to RGB, and read the associated mask from disk in grayscale mode
          image = cv2.imread(imagePath)
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          mask = cv2.imread(self.maskPaths[index]) #, 0)

          if self.transforms is not None:
            # apply the transformations to both image and its mask
            image = self.transforms(image)
            mask = self.transforms(mask)

          self.image_list.append(image)
          self.mask_list.append(mask)


        
        
    def __len__(self):
        # return the number of total samples contained in the dataset
        return len(self.imagePaths)
    
    
    def __getitem__(self, index): #idx
        # # grab the image path from the current index
        # imagePath = self.imagePaths[idx]
        
        # # load the image from disk, swap its channels from BGR to RGB, and read the associated mask from disk in grayscale mode
        # image = cv2.imread(imagePath)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # mask = cv2.imread(self.maskPaths[idx]) #, 0)
        
        # check to see if we are applying any transformations
        # if self.transforms is not None:
        #     # apply the transformations to both image and its mask
        #     image = self.transforms(image)
        #     mask = self.transforms(mask)
            
        # return a tuple of the image and its mask
        return (self.image_list[index], self.mask_list[index])

In [4]:
# mask = cv2.imread(trainMasks[0], 0)
# mask2 = cv2.imread(trainMasks[0])
# # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# plt.imshow(mask)


In [5]:
# plt.imshow(mask2)
# mask2.shape

# Data

## Data Preparation

In [6]:
# Load COCO annotations 
ann = '/content/Master/instances_val2017.json'

# Initialize the COCO api for instance annotations
coco = COCO(ann)

# Define the classes (out of the 81) which you want to see. Others will not be shown - just Person
filterClasses = ['person']

# Fetch class IDs only corresponding to the filterClasses
catIds = coco.getCatIds(catNms = filterClasses) 

# Get all images containing the above Category IDs
imgIds = coco.getImgIds(catIds = catIds)
print("Number of images containing all the  classes:", len(imgIds))


# Lists of Person Images and Masks
arr_images = []
arr_masks = []
for i in range (0, len(imgIds)):
    img = coco.loadImgs(imgIds[i])[0]
    arr_images.append((IMAGE_PATH + '/' + img['file_name']))
    arr_masks.append((MASK_PATH + '/' + img['file_name'][:12] + '.png'))

loading annotations into memory...
Done (t=1.04s)
creating index...
index created!
Number of images containing all the  classes: 2693


In [7]:
# Lists of images and masks 
imagePaths = arr_images
maskPaths = arr_masks

# Split data into train and test
TEST_SPLIT = 0.3

split = train_test_split(imagePaths, maskPaths, test_size = TEST_SPLIT, random_state = 19)

# Unpack the data split
(trainImages, testImages) = split[:2]
(trainMasks, testMasks) = split[2:]

# Write the testing image paths to disk so that we can use then when evaluating/testing our model
print("[INFO] saving testing image paths...")

f = open(TEST_PATHS, "w")
f.write("\n".join(testImages))
f.close()

[INFO] saving testing image paths...


## Datasets and Dataloaders

In [8]:
# Initialize training parameters

# TEST_SPLIT = 0.3

INIT_LR = 0.001
NUM_EPOCHS = 3
BATCH_SIZE = 16 

In [9]:
# define transformations
transforms_train = torchvision.transforms.Compose([transforms.ToPILImage(),
                                                   transforms.Resize((INPUT_IMAGE_HEIGHT, INPUT_IMAGE_WIDTH)),
                                                   transforms.ToTensor(),
                                                   transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                                                        std = [0.229, 0.224, 0.225])
                                                  ])

transforms_test = torchvision.transforms.Compose([transforms.ToPILImage(),
                                                  transforms.Resize((INPUT_IMAGE_HEIGHT, INPUT_IMAGE_WIDTH)),
                                                  transforms.ToTensor()
                                                 ])

# create the train and test datasets
trainDS = SegmentationDataset(imagePaths = trainImages, maskPaths = trainMasks, transforms = transforms_train)
testDS = SegmentationDataset(imagePaths = testImages, maskPaths = testMasks, transforms = transforms_test)


print(f"[INFO] found {len(trainDS)} examples in the training set...")
print(f"[INFO] found {len(testDS)} examples in the test set...")


# create the training and test data loaders
trainLoader = DataLoader(trainDS, 
                         shuffle = True,
                         batch_size = BATCH_SIZE, 
                         pin_memory = PIN_MEMORY
#                          num_workers = os.cpu_count()
                        )
testLoader = DataLoader(testDS, 
                        shuffle = False,
                        batch_size = BATCH_SIZE, 
                        pin_memory = PIN_MEMORY
#                         num_workers = os.cpu_count()
                       )

[INFO] found 1885 examples in the training set...
[INFO] found 808 examples in the test set...


## Configure Model

In [10]:
# https://github.com/qubvel/segmentation_models.pytorch
# https://github.com/qubvel/segmentation_models.pytorch#architectures
    
import segmentation_models_pytorch as smp

model = smp.Unet(
                    encoder_name = "resnet34",                    # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
                    encoder_weights = "imagenet",                 # use `imagenet` pre-trained weights for encoder initialization
                    in_channels = NUM_CHANNELS,                   # model input channels (1 for gray-scale images, 3 for RGB, etc.)
                    classes = NUM_CHANNELS,                        # model output channels (number of classes in your dataset) - person vs bg
                )

In [11]:
# initialize loss function and optimizer
lossFunc = nn.BCEWithLogitsLoss()
opt = Adam(model.parameters(), lr = INIT_LR)

# calculate steps per epoch for training and test set
trainSteps = len(trainDS) // BATCH_SIZE
testSteps = len(testDS) // BATCH_SIZE

# initialize a dictionary to store training history
H = {"train_loss": [], "test_loss": []}

### Train

In [12]:
# loop over epochs
print("[INFO] training the network...")

startTime = time.time()

for e in tqdm(range(NUM_EPOCHS)):
    
    # set the model in training mode
    model.train()
    
    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalTestLoss = 0
#     c = 0
    
    # loop over the training set
    for (i, (x, y)) in enumerate(trainLoader):
#         print(c + 1)
        
        # send the input to the device
        (x, y) = (x.to(device), y.to(device))
        
        # perform a forward pass and calculate the training loss
        pred = model(x)
        loss = lossFunc(pred, y)
        
        # first, zero out any previously accumulated gradients, then
        # perform backpropagation, and then update model parameters
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        # add the loss to the total training loss so far
        totalTrainLoss += loss
        
#         c = c + 1
        
    # switch off autograd
    with torch.no_grad():
        
        # set the model in evaluation mode
        model.eval()
        
        # loop over the validation set
        for (x, y) in testLoader:
            # send the input to the device
            (x, y) = (x.to(device), y.to(device))
            
            # make the predictions and calculate the validation loss
            pred = model(x)
            totalTestLoss += lossFunc(pred, y)
            
    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps
    avgTestLoss = totalTestLoss / testSteps
    
    # update our training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["test_loss"].append(avgTestLoss.cpu().detach().numpy())
    
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, NUM_EPOCHS))
    print("Train loss: {:.6f}, Test loss: {:.4f}".format(avgTrainLoss, avgTestLoss))
    
    
# display the total time needed to perform the training
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

[INFO] training the network...


  0%|          | 0/3 [03:48<?, ?it/s]


KeyboardInterrupt: ignored

In [85]:
(i, (x, y)) = next(enumerate(trainLoader))

In [89]:
y.shape

torch.Size([16, 3, 224, 224])